# Nettoyage des données

L'objectif de ce notebook est de nettoyer les données *train_data.csv* et *test_data.csv* afin de traiter les données manquantes et de supprimer les outliers (pour les données d'entraînement). Les données finales seront exportées dans *train_cleaned.csv* et *test_cleaned.csv*.

In [1]:
import pandas as pd # Pour la manipulation de tableaux de données
raw_train = pd.read_csv('Data/train_data.csv')

## Partie 0 : Analyse descriptive des données

In [2]:
import numpy as np # Pour les calculs mathématiques
import matplotlib.pyplot as plt # Pour les graphiques

import scipy.stats as scy # Pour les lois de probabilités et des tests statistiques
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor # Pour les VIF
from statsmodels.tools.tools import add_constant # Pour l'ajout d'une constante dans statsmodels
from statsmodels.stats.api import het_breuschpagan # Pour le test de Breusch-Pagan

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error # Pour les critères d'erreur

In [3]:
oldest_date = raw_train['date'].min()
newest_date = raw_train['date'].max()

print("Oldest date:", oldest_date)
print("Newest date:", newest_date)

raw_train.describe(include='all').transpose()

Oldest date: 2014-05-02T00:00:00Z
Newest date: 2015-05-27T00:00:00Z


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,17147.0,NaN,NaN,NaN,4605474711.841722,2879726234.50155,1000102.0,2126059294.5,3905120330.0,7339451560.0,9900000190.0
date,17147,369,2014-06-23T00:00:00Z,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prix,17147.0,NaN,NaN,NaN,543193.928034,371608.733909,75000.0,323000.0,450000.0,646000.0,7062500.0
nb_chambres,17147.0,NaN,NaN,NaN,3.371669,0.932029,0.0,3.0,3.0,4.0,33.0
nb_sdb,17147.0,NaN,NaN,NaN,2.12301,0.772906,0.0,1.75,2.25,2.5,7.75
m2_interieur,17147.0,NaN,NaN,NaN,193.732114,85.587837,26.941657,132.850242,178.372352,236.90078,932.738759
m2_jardin,17147.0,NaN,NaN,NaN,1411.998121,3879.062125,48.309179,468.227425,706.15013,992.985879,153414.994426
m2_etage,17147.0,NaN,NaN,NaN,166.652667,77.312579,26.941657,111.48272,145.856559,206.243032,823.114084
m2_soussol,17147.0,NaN,NaN,NaN,27.079448,41.221029,0.0,0.0,0.0,52.025269,447.788926
nb_etages,17147.0,NaN,NaN,NaN,1.497638,0.542015,1.0,1.0,1.5,2.0,3.5


In [4]:
train_corr = raw_train.drop(columns=['id', 'date']) # Statistiques Descriptives Bi-variées
round(train_corr.corr(), 2)

,prix,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,vue_note,etat_note,design_note,annee_construction,annee_renovation,m2_interieur_15voisins,m2_jardin_15voisins,zipcode,lat,long
prix,1.00,0.31,0.53,0.70,0.09,0.61,0.32,0.25,0.27,0.40,0.03,0.67,0.05,0.13,0.58,0.08,-0.05,0.31,0.02
nb_chambres,0.31,1.00,0.52,0.58,0.03,0.47,0.31,0.17,-0.01,0.08,0.03,0.36,0.15,0.03,0.39,0.03,-0.16,-0.01,0.13
nb_sdb,0.53,0.52,1.00,0.75,0.08,0.68,0.28,0.50,0.06,0.19,-0.13,0.67,0.51,0.05,0.57,0.08,-0.20,0.02,0.22
m2_interieur,0.70,0.58,0.75,1.00,0.17,0.88,0.43,0.35,0.10,0.28,-0.07,0.77,0.32,0.06,0.76,0.18,-0.20,0.05,0.24
m2_jardin,0.09,0.03,0.08,0.17,1.00,0.18,0.02,-0.01,0.02,0.07,-0.01,0.11,0.05,0.00,0.15,0.70,-0.13,-0.08,0.23
m2_etage,0.61,0.47,0.68,0.88,0.18,1.00,-0.06,0.52,0.07,0.17,-0.16,0.76,0.42,0.02,0.73,0.19,-0.26,-0.00,0.34
m2_soussol,0.32,0.31,0.28,0.43,0.02,-0.06,1.00,-0.25,0.08,0.27,0.17,0.17,-0.13,0.08,0.20,0.02,0.08,0.11,-0.15
nb_etages,0.25,0.17,0.50,0.35,-0.01,0.52,-0.25,1.00,0.02,0.03,-0.27,0.46,0.49,0.00,0.28,-0.01,-0.06,0.05,0.12
vue_mer,0.27,-0.01,0.06,0.10,0.02,0.07,0.08,0.02,1.00,0.42,0.02,0.08,-0.03,0.10,0.08,0.03,0.04,-0.01,-0.04
vue_note,0.40,0.08,0.19,0.28,0.07,0.17,0.27,0.03,0.42,1.00,0.05,0.25,-0.06,0.10,0.28,0.07,0.09,0.01,-0.08


## Partie 1 : Mise en forme des données

In [5]:
taille_initiale = len(raw_train)

def perte(set): # Fonction pour calculer la perte de données par rapport à l'ensemble initial
    prop_perte = (taille_initiale - len(set)) / taille_initiale
    print("Perte actuelle de", round(prop_perte*100, 2), "%")

### Suppression des champs incomplets

In [6]:
## Missing or zero values and percentage

# arguments qui doivent avoir une valeur non nulle
arguments1 = ["prix", "nb_chambres", "nb_sdb", "m2_interieur", "nb_etages", "etat_note", "design_note", "annee_construction", "m2_interieur_15voisins", "zipcode", "lat", "long"]

# arguments qui peuvent avoir une valeur nulle
arguments2 = ["m2_jardin", "m2_etage", "m2_soussol", "vue_note", "vue_mer", "annee_renovation", "m2_jardin_15voisins"]


for argument in arguments1:
    manquant = raw_train[argument].isna().sum()+raw_train[argument].isin([0]).sum()
    if manquant > 0:
        print("\033[1m\033[4m", argument, "\033[0m")
        print("Nombre de valeurs manquantes ou nulles pour", argument, ":", manquant)
        print("Propotion de valeurs manquantes ou nulles pour", argument, ":", manquant/len(raw_train)*100, "%")

for argument in arguments2:
    manquant = raw_train[argument].isna().sum()
    if manquant > 0:
        print("\033[1m\033[4m", argument, "\033[0m")
        print("Nombre de valeurs manquantes pour", argument, ":", manquant)
        print("Propotion de valeurs manquantes pour", argument, ":", manquant/len(raw_train)*100, "%")

 nb_chambres 
Nombre de valeurs manquantes ou nulles pour nb_chambres : 9
Propotion de valeurs manquantes ou nulles pour nb_chambres : 0.05248731556540502 %
 nb_sdb 
Nombre de valeurs manquantes ou nulles pour nb_sdb : 7
Propotion de valeurs manquantes ou nulles pour nb_sdb : 0.04082346766198169 %


In [7]:
# Nb Chambres et Nb Sdb : on drop les valeurs abhérentes

no_missing_train = raw_train.dropna(subset=['nb_chambres']).loc[raw_train['nb_chambres'] != 0] #on les drop car -5% sans données
no_missing_train = no_missing_train.dropna(subset=['nb_sdb']).loc[no_missing_train['nb_sdb'] != 0] #on les drop car -5% sans données

perte(no_missing_train)

def is_cleaned(train):
    flag = True
    for argument in arguments1:
        manquant = train[argument].isna().sum()+train[argument].isin([0]).sum()
        if manquant != 0:
            print("Nettoyage des données non satisfaisant pour", argument)
            flag = False
            

    for argument in arguments2:
        manquant = train[argument].isna().sum()
        if manquant != 0:
            print("Nettoyage des données non satisfaisant pour", argument)
            flag = False
    if flag:
        print("Nettoyage des données satisfaisant")

is_cleaned(no_missing_train)

Perte actuelle de 0.07 %
Nettoyage des données satisfaisant


In [8]:
no_missing_train.to_csv('Data/train_cleaned_w_outlier.csv', index=False)

### Identification et suppression des outliers

La réduction de cardinalité n'est pas considérée ici car toutes les valeurs sont numériques (on ne va pas créer une catégorie "plus que 6" chambres car à ce moment, par quelle valeur numérique la remplacer ?)

In [9]:
arguments = arguments1 + arguments2

def count_values(arguments=arguments, champs="None"):
    if champs != "None":
        print("\033[1m\033[4m", champs, "\033[0m")
        print(train[champs].value_counts())
    else:
        for argument in arguments:
            print("\033[1m\033[4m", argument, "\033[0m")
            print(train[argument].value_counts())

#### Détermination des outliers par méthode des seuils
On fixe le seuil à 1% pour chaques variables.

In [10]:
#Catégorie Chambre

prop_sup_6_chambres = len(no_missing_train[no_missing_train['nb_chambres'] >= 7])/len(no_missing_train)*100
print(prop_sup_6_chambres, "% des maisons ont plus de 7 chambres")

id_outlier_chambres = no_missing_train[no_missing_train['nb_chambres'] >= 7].index #on les drop car -5% sans données

id_outlier_total = id_outlier_chambres
perte(no_missing_train.drop(id_outlier_total))

0.2684563758389262 % des maisons ont plus de 7 chambres
Perte actuelle de 0.34 %


In [11]:
# Catégorie SdB

nb_occurrences = no_missing_train['nb_sdb'].value_counts()
count = sum(nb_occurrences[nb_occurrences <= 60])
print(count/len(no_missing_train)*100, "% des maisons qui ont un nombre de salle de bain qui apparaît <=60 fois dans le jeu de données")

id_outlier_sdb = no_missing_train[no_missing_train['nb_sdb'].map(no_missing_train['nb_sdb'].value_counts()) <= 60].index #on les drop car -5% sans données

id_outlier_total = id_outlier_total.union(id_outlier_sdb)
perte(no_missing_train.drop(id_outlier_total))

0.8345491683688357 % des maisons qui ont un nombre de salle de bain qui apparaît <=60 fois dans le jeu de données
Perte actuelle de 1.14 %


In [12]:
#Catégorie vue_note : on ne touche à rien (peu de valeurs, pas de valeurs abhérentes)

# Catégorie etat_note

prop_etat_note_2 = len(no_missing_train[no_missing_train['etat_note'] <= 2]) / len(no_missing_train) * 100
print("Proportion of etat_note <= 2:", prop_etat_note_2, "%")

#On drop ces valeurs
id_outlier_etat_note = no_missing_train[no_missing_train['etat_note'] <= 2].index

id_outlier_total = id_outlier_total.union(id_outlier_etat_note)
perte(no_missing_train.drop(id_outlier_total))

Proportion of etat_note <= 2: 0.8929092500729501 %
Perte actuelle de 2.01 %


In [13]:
# Proportion de maisons avec m2_interieur > 450
prop_sup_450 = len(no_missing_train[no_missing_train['m2_interieur'] > 500])/len(no_missing_train)*100
print(prop_sup_450, "% des maisons ont un m2_interieur > 470")

# Proportion de maisons avec m2_interieur < 50
prop_inf_50 = len(no_missing_train[no_missing_train['m2_interieur'] < 60])/len(no_missing_train)*100
print(prop_inf_50, "% des maisons ont un m2_interieur < 65")

id_outlier_m2_interieur = no_missing_train[no_missing_train['m2_interieur'] > 500].index.union(no_missing_train[no_missing_train['m2_interieur'] < 60].index)

id_outlier_total = id_outlier_total.union(id_outlier_m2_interieur)
perte(no_missing_train.drop(id_outlier_total))

0.665304931426904 % des maisons ont un m2_interieur > 470
0.39101254741756636 % des maisons ont un m2_interieur < 65
Perte actuelle de 2.71 %


In [14]:
prop_sup_3_etages = len(no_missing_train[no_missing_train['nb_etages'] > 3]) / len(no_missing_train) * 100
print("Proportion of properties with more than 3 floors:", prop_sup_3_etages, "%")

id_outlier_etages = no_missing_train[no_missing_train['nb_etages'] > 3].index

id_outlier_total = id_outlier_total.union(id_outlier_etages)
perte(no_missing_train.drop(id_outlier_total))

Proportion of properties with more than 3 floors: 0.02918004085205719 %
Perte actuelle de 2.74 %


In [15]:
# Catégorie design_note

prop_design_note = len(no_missing_train[(no_missing_train['design_note'] >= 13) | (no_missing_train['design_note'] <= 4)]) / len(no_missing_train) * 100
print("Proportion of properties with design_note >= 13 or <= 4:", prop_design_note, "%")

id_outlier_design_note = no_missing_train[(no_missing_train['design_note'] >= 13) | (no_missing_train['design_note'] <= 4)].index

id_outlier_total = id_outlier_total.union(id_outlier_design_note)
perte(no_missing_train.drop(id_outlier_total))

Proportion of properties with design_note >= 13 or <= 4: 0.21593230230522323 %
Perte actuelle de 2.79 %


In [16]:
prop_sup_20000 = len(no_missing_train[no_missing_train['m2_jardin'] > 20000]) / len(no_missing_train) * 100
print(prop_sup_20000, "% des maisons ont un m2_jardin > 20000")

id_outlier_jardin = no_missing_train[no_missing_train['m2_jardin'] > 20000].index

id_outlier_total = id_outlier_total.union(id_outlier_jardin)
perte(no_missing_train.drop(id_outlier_total))

0.9512693317770644 % des maisons ont un m2_jardin > 20000
Perte actuelle de 3.66 %


In [17]:
prop_sup_175 = len(no_missing_train[no_missing_train['m2_soussol'] > 160]) / len(no_missing_train) * 100
print(prop_sup_175, "% des maisons ont un m2_soussol > 160")

id_outlier_soussol = no_missing_train[no_missing_train['m2_soussol'] > 160].index

id_outlier_total = id_outlier_total.union(id_outlier_soussol)
perte(no_missing_train.drop(id_outlier_total))

0.8578932010504815 % des maisons ont un m2_soussol > 160
Perte actuelle de 4.25 %


In [18]:
# Proportion de maisons avec m2_jardin_15voisins > 10000
prop_sup_10000 = len(no_missing_train[no_missing_train['m2_jardin_15voisins'] > 20000]) / len(no_missing_train) * 100
print(prop_sup_10000, "% des maisons ont un m2_jardin_15voisins > 10000")

# Proporiton de maisons avec m2_jardin_15voisins < 100
prop_inf_100 = len(no_missing_train[no_missing_train['m2_jardin_15voisins'] < 100]) / len(no_missing_train) * 100
print(prop_inf_100, "% des maisons ont un m2_jardin_15voisins < 100")

id_outlier_jardin_15voisins = no_missing_train[no_missing_train['m2_jardin_15voisins'] > 20000].index.union(no_missing_train[no_missing_train['m2_jardin_15voisins'] < 100].index)

id_outlier_total = id_outlier_total.union(id_outlier_jardin_15voisins)
perte(no_missing_train.drop(id_outlier_total))

0.5252407353370294 % des maisons ont un m2_jardin_15voisins > 10000
0.42602859644003505 % des maisons ont un m2_jardin_15voisins < 100
Perte actuelle de 4.81 %


Les critères manquant n'avaient pas d'outlier pertinent à écarter

In [19]:
no_outlier_train = no_missing_train.drop(id_outlier_total)
no_outlier_train.to_csv('Data/train_cleaned_no_outlier.csv', index=False)

## Préparation des données de test

In [23]:
# Chargement des données de test
raw_test = pd.read_csv('Data/test_data.csv')

## Missing or zero values and percentage
def missing_values_test(raw_test=raw_test):
    # arguments qui doivent avoir une valeur non nulle
    flag = True
    arguments1 = ["nb_chambres", "nb_sdb", "m2_interieur", "nb_etages", "etat_note", "design_note", "annee_construction", "m2_interieur_15voisins", "zipcode", "lat", "long"]

    # arguments qui peuvent avoir une valeur nulle
    arguments2 = ["m2_jardin", "m2_etage", "m2_soussol", "vue_note", "vue_mer", "annee_renovation", "m2_jardin_15voisins"]


    for argument in arguments1:
        manquant = raw_test[argument].isna().sum()+raw_test[argument].isin([0]).sum()
        if manquant > 0:
            print("\033[1m\033[4m", argument, "\033[0m")
            print("Nombre de valeurs manquantes ou nulles pour", argument, ":", manquant)
            print("Propotion de valeurs manquantes ou nulles pour", argument, ":", manquant/len(raw_test)*100,"%")
            flag = False

    for argument in arguments2:
        manquant = raw_test[argument].isna().sum()
        if manquant > 0:
            print("\033[1m\033[4m", argument, "\033[0m")
            print("Nombre de valeurs manquantes pour", argument, ":", manquant)
            print("Propotion de valeurs manquantes pour", argument, ":", manquant/len(raw_test)*100, "%")
            flag = False
    
    if flag:
        print("No missing values")

missing_values_test()

 nb_chambres 
Nombre de valeurs manquantes ou nulles pour nb_chambres : 4
Propotion de valeurs manquantes ou nulles pour nb_chambres : 0.09330534173081409 %
 nb_sdb 
Nombre de valeurs manquantes ou nulles pour nb_sdb : 3
Propotion de valeurs manquantes ou nulles pour nb_sdb : 0.06997900629811056 %


In [24]:
test_cleaned = raw_test.copy()

test_cleaned['nb_chambres'] = test_cleaned['nb_chambres'].replace(0, test_cleaned['nb_chambres'].mean())
test_cleaned['nb_sdb'] = test_cleaned['nb_sdb'].replace(0, test_cleaned['nb_sdb'].mean())

missing_values_test(test_cleaned)

No missing values


In [25]:
# On rend les m2_interieur, m2_interieur_15voisins, m2_jardin, m2_etage, m2_soussol, m2_jardin_15voisins entiers

test_cleaned['m2_interieur'] = test_cleaned['m2_interieur'].astype(int)
test_cleaned['m2_interieur_15voisins'] = test_cleaned['m2_interieur_15voisins'].astype(int)
test_cleaned['m2_jardin'] = test_cleaned['m2_jardin'].astype(int)
test_cleaned['m2_etage'] = test_cleaned['m2_etage'].astype(int)
test_cleaned['m2_soussol'] = test_cleaned['m2_soussol'].astype(int)
test_cleaned['m2_jardin_15voisins'] = test_cleaned['m2_jardin_15voisins'].astype(int)

In [26]:
test_cleaned.to_csv('Data/test_cleaned.csv', index=False)